In [1]:
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
# !pip install -q -U tensorboardX
!pip install -q wandb
# !pip install -q -U torchvision
!pip install -q -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.1/343.1 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 6.1 MB/s eta 0:00:00


In [2]:
import torch

torch.cuda.is_available()

False

In [ ]:
from enum import Enum
from functools import partial
import pandas as pd
import torch
import json

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, set_seed
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
from peft import LoraConfig, TaskType

seed = 42
set_seed(seed)

import os

token = "" # the token should have write access

In [4]:
dataset_name = "NousResearch/hermes-function-calling-v1"
dataset = load_dataset(dataset_name, token=token)

README.md:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

func-calling-singleturn.json:   0%|          | 0.00/15.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1893 [00:00<?, ? examples/s]

In [5]:
import re

def clean_system_prompt(prompt: str) -> str:
    prompt = re.sub(r"</?tools>\n?", "", prompt)

    prompt = re.sub(r"</?tool_call>\n?", "", prompt)

    prompt = re.sub(r"within\s+XML\s+tags", "", prompt, flags=re.IGNORECASE)
    prompt = re.sub(r"within\s+tags", "", prompt, flags=re.IGNORECASE)

    prompt = re.sub(r"\s{2,}", " ", prompt)
    prompt = re.sub(r"\s+([.,!?])", r"\1", prompt)

    return prompt.strip()

def clean_response(prompt: str) -> str:
    prompt = re.sub(r"</?tools>\n?", "", prompt)

    prompt = re.sub(r"</?tool_call>\n?", "", prompt)

    return prompt.strip()


def preprocess(sample):
    messages = sample['conversations']
    system_prompt = None
    user_prompt = None
    assistant_response = None

    for d in messages:
        for key in d:
            if d[key] == 'human':
                d[key] = 'user'
            elif d[key] == 'gpt':
                d[key] = 'assistant'

        if 'from' in d:
            d['role'] = d.pop('from')
        if 'value' in d:
            d['content'] = d.pop('value')

    for d in messages:
        if d['role'] == 'system':
            system_prompt = d
        elif d['role'] == 'user':
            user_prompt = d
        elif d['role'] == 'assistant':
            assistant_response = d

    system_prompt['content'] = clean_system_prompt(system_prompt['content'])
    assistant_response['content'] = clean_response(assistant_response['content'])

    prompt = [system_prompt, user_prompt]
    completion = [assistant_response]

    return {"prompt": prompt, "completion": completion}


In [6]:
dataset_preprocessed = dataset.map(preprocess, remove_columns=['conversations', 'task', 'category', 'id', 'subcategory'])
dataset_preprocessed = dataset_preprocessed['train'].train_test_split(test_size=0.1)
dataset_preprocessed

Map:   0%|          | 0/1893 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 1703
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 190
    })
})

In [7]:
model_name = "meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token=token)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype="auto", token=token)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [8]:
prompt = dataset_preprocessed['train']['prompt'][10]
print(tokenizer.apply_chat_template(prompt, tokenize=False))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a function calling AI model. You are provided with function signatures. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.
[{'type': 'function', 'function': {'name': 'add_patient_record', 'description': 'Adds a new patient record to the EMR system with personal and medical information.', 'parameters': {'type': 'object', 'properties': {'patient_info': {'type': 'object', 'description': 'Personal information of the patient.', 'properties': {'name': {'type': 'string', 'description': 'Full name of the patient.'}, 'date_of_birth': {'type': 'string', 'description': 'Date of birth of the patient.'}, 'address': {'type': 'string', 'description': 'Residential address of the patient.'}, 'contact': {'type': 'string', 'description': 'Contact number of the patient.'}}, 'required': ['name', 'd

In [9]:
from peft import LoraConfig

# TODO: Configure LoRA parameters
# r: rank dimension for LoRA update matrices (smaller = more compression)
rank_dimension = 16
# lora_alpha: scaling factor for LoRA layers (higher = stronger adaptation)
lora_alpha = 64
# lora_dropout: dropout probability for LoRA layers (helps prevent overfitting)
lora_dropout = 0.05

peft_config = LoraConfig(r=rank_dimension,
                         lora_alpha=lora_alpha,
                         lora_dropout=lora_dropout,
                         target_modules=["gate_proj","q_proj","o_proj","k_proj","down_proj","up_proj","v_proj"], # wich layer in the transformers do we target ?
                         task_type=TaskType.CAUSAL_LM)

In [ ]:
username="maxkochanoff" # REPLACE with your Hugging Face username
output_dir = "llama-3.1-function-calling-hermes" # The directory where the trained model checkpoints, logs, and other artifacts will be saved. It will also be the default name of the model when pushed to the hub if not redefined later.
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
gradient_accumulation_steps = 32
logging_steps = 5
learning_rate = 1e-4 # The initial learning rate for the optimizer.

max_grad_norm = 1.0
num_train_epochs = 1
warmup_ratio = 0.1
lr_scheduler_type = "cosine"

training_arguments = SFTConfig(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_strategy="no",
    eval_strategy="epoch",
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    weight_decay=0.1,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    bf16=True,
    hub_private_repo=False,
    push_to_hub=False,
    num_train_epochs=num_train_epochs,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    packing=False,
    completion_only_loss=True,
    # max_seq_length=model.config.max_position_embeddings,
    max_seq_length=4096,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset_preprocessed["train"],
    eval_dataset=dataset_preprocessed["test"],
    processing_class=tokenizer,
    peft_config=peft_config,
)

In [ ]:
trainer.train_dataset

When I used bigger context window size, hist was the same => this is the original distribution

In [ ]:
import matplotlib.pyplot as plt

def compute_length(example):
    return {"num_tokens": len(example["input_ids"])}

dataset_with_lengths = trainer.train_dataset.map(compute_length)

plt.hist(dataset_with_lengths['num_tokens'], bins=10)
plt.show()

In [ ]:
dl = trainer.get_train_dataloader()
batch = next(iter(dl))

In [ ]:
batch

In [ ]:
trainer.train()
trainer.save_model()